In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [72]:
import openai
import pandas as pd
import random
import time
# Set your API key
openai.api_key = ''

# Load the CSV file
file_path = "/content/sample_data/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv"
data = pd.read_csv(file_path)

# Define SDOH labels and adverse categories
sdoh_labels = ['HOUSING', 'TRANSPORTATION', 'RELATIONSHIP', 'PARENT', 'EMPLOYMENT', 'SUPPORT']
adverse_categories = ['adverse', 'non-adverse']

# Function to create a prompt for the ChatGPT API to expand the dialogue
def create_expansion_prompt(dialogue, label, adverse):
    return f"Given that this scenario is classified as '{adverse}',Keep the existing dialogue same do change the content. Then please add the medical dialogue with additional content focused on the SDOH category'{label} There should be only one output with both Adverse Label and SDOH Label combined':\n{dialogue}"

# Function to create a prompt for the ChatGPT API to summarize the dialogue
def create_summary_prompt(dialogue, label, adverse):
    return f"Create a comprehensive summary incorporating the newly added '{label}' content, and the '{adverse} the dialogue given in maximum of 80 words.' context:\nDialogue: {dialogue}"

# Function to call the ChatGPT API
def call_chatgpt_api(prompt):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # Ensure this model is available in your plan
        messages =[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=1000
    )
    print(response)
    mess=response.choices[0].message.content
    return mess

# Iterate over each dialogue in the CSV
for index, row in data.iterrows():
    original_dialogue = row['dialogue']
    existing_summary = row['section_text']
    # Randomly choose an adverse category and an SDOH label for each dialogue
    adverse = random.choice(adverse_categories)
    label = random.choice(sdoh_labels)

    # Create the prompt to expand the dialogue
    expansion_prompt = create_expansion_prompt(original_dialogue, label, adverse)
    # Send the prompt to the ChatGPT API and get the expanded dialogue
    expanded_dialogue = call_chatgpt_api(expansion_prompt)

    # Create the prompt to generate an updated summary
    summary_prompt = create_summary_prompt(expanded_dialogue, label, adverse)
    # Send the prompt to the ChatGPT API and get the updated summary
    updated_summary = call_chatgpt_api(summary_prompt)

    # Update the dialogue, summary, and labels in the dataframe
    data.at[index, 'dialogue'] = expanded_dialogue
    data.at[index, 'section_text'] = updated_summary
    data.at[index, 'SDOH label'] = label
    data.at[index, 'Adverse Category'] = adverse

    if (index + 1) % 30 == 0:
        data.to_csv('updated_mts_dialog_testset.csv', index=False)
        print(f"Data saved after processing {index + 1} entries. Pausing for a minute.")
        time.sleep(60)

# Save the updated dataframe to a new CSV file
updated_file_path = 'updated_mts_dialog_testset.csv'
data.to_csv(updated_file_path, index=False)


ChatCompletion(id='chatcmpl-9E07Bi4SKkscjohuRoqRTDeCc6FCi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Doctor: Good afternoon, sir. Did you just have a birthday? I don't have my chart with me right now, the nurse is bringing it. \nPatient: Good afternoon, sir. Yes, I just turned fifty five. \nDoctor: You identify as African American, correct? \nPatient: Yes, that's right. \nDoctor: When was your last visit, sir? \nPatient: Um, it was on July twenty ninth two thousand eight. \nDoctor: Yes, I see. Did we go over your M R I results? \nPatient: No, I was having those new seizures, remember?\nDoctor: Yes, I do. Well, the M R I demonstrated right contrast temporal mass. \nPatient: What exactly does that mean, doctor? \nDoctor: Well, given this mass, and your new seizures, I am concerned that this could be a high grade glioma, we'll need to do more tests.\n\n[Medical Dialogue - SDOH]\nDoctor: In light of these findings, I also want to 

In [ ]:
print(data)
updated_file_path = 'updated_mts_dialog_testset.csv'
data.to_csv(updated_file_path, index=False)

      ID section_header                                       section_text  \
0      0          GENHX  The 55-year-old African-American male patient,...   
1      1      FAM/SOCHX  The patient reported a family history of strok...   
2      2            ROS  Doctor: Any pain in your muscles?  \nPatient: ...   
3      3      FAM/SOCHX  Doctor: Anyone in your family facing challenge...   
4      4      FAM/SOCHX  Patient's father passed away from a thoracic a...   
..   ...            ...                                                ...   
195  195          GENHX  Briefly, the patient is a very pleasant 72-yea...   
196  196            ROS  NOSE AND THROAT: Negative postnasal drip, nega...   
197  197          GENHX  The patient is alert and oriented x3 and sitti...   
198  198  PASTMEDICALHX              Past medical history is unremarkable.   
199  199  IMMUNIZATIONS  Immunizations were up to date for influenza, n...   

                                              dialogue    SDOH 